In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

#### Number of records and attributes

In [ ]:
## read data
data = pd.read_csv('orders_train.txt', sep=';')
print('Number of records between {} to {}: {}'.format(df['orderDate'].iloc[0], df['orderDate'].iloc[-1], len(data)))
print('Number of attributes corresponding to a single record {}:'.format(data.shape[1]-1))
data.head()

Number of records between 2014-01-01 to nan: 2116247
Number of attributes corresponding to a single record 14:


,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity
0,a1000001,2014-01-01,i1000382,1972.0,44,3.0,1.0,10.00,29.99,0,0.0,c1010575,2.0,BPRG,0.0
1,a1000001,2014-01-01,i1000550,3854.0,44,3.0,1.0,20.00,39.99,0,0.0,c1010575,2.0,BPRG,0.0
2,a1000002,2014-01-01,i1001991,2974.0,38,8.0,1.0,35.00,49.99,0,0.0,c1045905,4.0,BPRG,0.0
3,a1000002,2014-01-01,i1001999,1992.0,38,8.0,1.0,49.99,49.99,0,0.0,c1045905,4.0,BPRG,1.0
4,a1000003,2014-01-01,i1001942,1968.0,42,8.0,1.0,10.00,35.99,0,0.0,c1089295,2.0,PAYPALVC,0.0


#### Data types and statistics

In [ ]:
print(data.info())

#### Missing values and inconsistent records

In [ ]:
#check NA values
print(data.isna().sum())
#drop NA values since not a significant number of records contain NA
data = data.dropna()

orderID             0
orderDate           1
articleID           1
colorCode           1
sizeCode            1
productGroup      328
quantity            1
price               1
rrp               328
voucherID           5
voucherAmount       1
customerID          1
deviceID            1
paymentMethod       1
returnQuantity      1
dtype: int64


In [ ]:
## inconsistent records
drop_idx = data[(data['quantity']==0) |  (data['price']==0) | (data['quantity']<data['returnQuantity']) ].index
data = data.drop(drop_idx)
data.shape

#### Temporal analysis

In [ ]:
# make new columns for year, month and day
data[['year', 'month', 'day']] = data['orderDate'].str.split('-', expand=True)
monthlySale = data.groupby(['month', 'day', 'year'])['quantity'].sum().reset_index()
monthlyReturn = data.groupby(['month', 'day', 'year'])['returnQuantity'].sum().reset_index()
# visualize montly sale and return for 2014 and 2015
plt.figure(figsize=(14, 8))
sns.swarmplot(x='month', y='quantity', data=monthlySale, hue = 'year', dodge=True)
sns.boxplot(x='month', y='quantity', data=monthlySale, hue = 'year', dodge=True)
plt.show()

plt.figure(figsize=(14, 8))
sns.swarmplot(x='month', y='returnQuantity', data=monthlyReturn, hue = 'year', dodge=True)
sns.boxplot(x='month', y='returnQuantity', data=monthlyReturn, hue = 'year')
plt.show()

In [ ]:
# visualizing the average discounts for different product groups in each month
discount = data.groupby(['productGroup', 'month'])['discountRatio'].mean().reset_index()
plt.figure(figsize= (10,5))
sns.lineplot(data= discount, x ='month', y ='discountRatio', hue=discount['productGroup'].astype(str), marker='o')
plt.legend(fontsize=8)
plt.ylabel('Average discount (%)')

#### Product characteristics- Sizes offered by the retailer

In [ ]:
sizes = data['sizeCode'].value_counts()
sns.barplot(x= sizes.index, y=sizes.values, hue = sizes.index)
plt.xticks(rotation=45)
plt.ylabel('quantity')
plt.show()

In [ ]:
# dropping sizes A and I for now...
data = data[~data['sizeCode'].isin(['A', 'I'])]
# sizes bought by unique customers
custm_size = data.groupby('customerID')['sizeCode'].agg(list).reset_index()
custm_size['sizeCode'] = custm_size['sizeCode'].apply(lambda x: list(set(x)))
custm_size

In [ ]:
#encode the data
encoder = TransactionEncoder() #initialize the encoder
ecd_data = encoder.fit(custm_size['sizeCode']).transform(custm_size['sizeCode'])# encoded data

# make dataframe of encoded data
ecd_df = pd.DataFrame(ecd_data, columns=encoder.columns_)
FP = apriori(ecd_df, min_support= 0.00005, use_colnames=True, max_len = 2)
rules = association_rules(FP, metric='confidence', min_threshold= 0.001)


In [ ]:
rules['antecedents'] = list(map(lambda x: list(x)[0], rules['antecedents']))
rules['consequents'] = list(map(lambda x: list(x)[0], rules['consequents']))
bold = "\033[1m"
reset = "\033[0m"
print(bold+'Mapping of other units of sizes to standard sizes between 34 - 44:\n'+reset)
print(bold+'other unit\t standard unit \t confidence'+reset)
for size in ['75', '80', '85', '90', '95', '100', 'XS', 'S', 'M', 'L', 'XL', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33']:
    idx = [rules[rules['antecedents'] == size]['confidence'].idxmax()]
    print(rules.iloc[idx]['antecedents'].to_string(index=False), '\t\t', rules.iloc[idx]['consequents'].to_string(index=False),
          '\t\t', rules.iloc[idx]['confidence'].to_string(index=False))


In [ ]:
mapping = {'75':'34', '80': '36', '85': '38', '90': '38', '95': '40', '100': '42', 'XS':'34', 'S': '36',  'M': '38','L':'42',
'XL':'42', '24':'34', '25':'34' , '26':'36', '27':'36', '28':'38','29': '38','30':'40','31':'40','32':'42', '33':'42'}
for key in mapping.keys():
    idx = data['sizeCode']==key
    data.loc[idx, 'sizeCode'] = mapping[key]
# new size distribution after mapping different sizes into standard sizes
sizes = data['sizeCode'].value_counts()
sns.barplot(x= sizes.index, y=sizes.values, hue=sizes.index)
plt.ylabel('quantity')
plt.show()

#### Buying and return behavior of customers w.r.t. product characterictics

In [ ]:
#visualize the sizes that are most often bought
sizes_bought = data.groupby('sizeCode')['quantity'].apply(lambda x: sum(list(x))).reset_index()
sizes_bought['%'] = sizes_bought['quantity']/data['quantity'].sum()
plt.pie(x = sizes_bought['%'], labels = sizes_bought['sizeCode'], autopct='%1.1f%%')
plt.title('Distribution of sizes bought by customers')
plt.show()
del sizes_bought

In [ ]:
#visualize the product group that are most often bought
category_bought = data.groupby('productGroup')['quantity'].apply(lambda x: sum(list(x))).reset_index()
category_bought['%'] = category_bought['quantity']/data['quantity'].sum()
category_bought = category_bought.sort_values(by = '%', ascending = False).iloc[0:10]
plt.figure(figsize=(6,6))
plt.pie(x = category_bought['%'], labels = category_bought['productGroup'], autopct='%1.1f%%')
plt.title('Distribution of prodduct group bought by customers')
plt.show()
del category_bought

In [ ]:
# colors offered in top selling items
print('Total number of unique color codes:', data['colorCode'].nunique())
colorcode = data.groupby('articleID')['quantity'].sum().sort_values(ascending= False).reset_index().head(20)\
.merge(data.groupby('articleID')['colorCode'].nunique().sort_values(ascending= False).reset_index().rename(columns={'colorCode': 'uniqueColorCodes'}).head(20), on ='articleID')

plt.scatter(x=colorcode['articleID'], y = colorcode['quantity'], s = 15*colorcode['uniqueColorCodes'], c = np.random.rand(len(colorcode)), \
            marker='p', alpha=0.7, cmap  = plt.get_cmap("rainbow"))
plt.ylabel('Articles sold')
plt.xlabel('Article ID')
plt.title('Number of colors offered in top selling articles ')
plt.gca().set_facecolor(color='seashell')
plt.xticks(rotation=45)
plt.show()

colorcode

#### Distribution of number of items in an order

In [ ]:
uniqueOrders = data.groupby('orderID')[['quantity', 'returnQuantity']].sum().reset_index()
fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.8, 1.3])
axes2 = fig.add_axes([0.35, 0.55, 0.4, 0.8])
axes.hist(uniqueOrders['quantity'], bins=range(0, max(uniqueOrders['quantity']), 2), align='left', edgecolor='black', alpha=0.7)
axes2.hist(uniqueOrders['quantity'], bins=range(25, max(uniqueOrders['quantity']), 2), align='left', edgecolor='black', alpha=0.7)
axes2.set_facecolor('lightgray')
plt.xlabel('Number of items in an order')
plt.ylabel('Count')
plt.title('Distribution of number of items in an order')
fig.show()


In [ ]:
#add a binary return variable
data['bin_return'] = data['returnQuantity'].apply(lambda x: int(x!=0))
data.loc[data['returnQuantity']>1].head()

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity,bin_return
958,a1000321,2014-01-01,i1000314,1493.0,40,3.0,2.0,45.98,22.99,0,0.0,c1089417,2.0,BPRG,2.0,1
1056,a1000350,2014-01-01,i1000312,1000.0,44,3.0,2.0,35.98,17.99,0,0.0,c1022255,3.0,BPRG,2.0,1
1057,a1000350,2014-01-01,i1000325,1000.0,44,3.0,2.0,39.98,19.99,0,0.0,c1022255,3.0,BPRG,2.0,1
1058,a1000350,2014-01-01,i1000663,1000.0,44,3.0,2.0,35.98,17.99,0,0.0,c1022255,3.0,BPRG,2.0,1
1396,a1000450,2014-01-02,i1000326,1961.0,42,3.0,2.0,39.98,19.99,0,0.0,c1089473,2.0,CBA,2.0,1


# Making the "order_df" dataframe

In [ ]:
def list_items(vec):
    lst = []
    for item in vec:
        lst.append(item)
    return lst

In [ ]:
uniq_order_df = data.groupby('orderID', as_index=False).agg({'orderDate': list_items,
                                                            'articleID': list_items,
                                                            'colorCode': list_items,
                                                            'productGroup': list_items,
                                                            'quantity': np.sum,
                                                            'price': np.sum,
                                                            'rrp': np.sum,
                                                            'voucherID': list_items,
                                                            'voucherAmount': list_items,
                                                            'customerID': list_items,
                                                            'paymentMethod': list_items,
                                                            'returnQuantity': np.sum})
uniq_order_df.head()

In [ ]:
# check if all the lists are homogeneous (all elements are equal)
def homogeneous(lst):
    first = lst[0]
    for item in lst:
        if item != first:
            return 0
    return 1

num = 1
list_columns = uniq_order_df[['orderDate', 'voucherID', 'voucherAmount', 'customerID', 'paymentMethod']]
for i in range(list_columns.shape[0]):
    row = list_columns.iloc[i]
    for col in row.values:
        num *= homogeneous(col)

print(num)

Since `num`=1, we know that all these lists are homogeneous, thus we do not need to store these row values as lists, rather we can store them as strings. Thus, we make a new dataframe.

In [ ]:
uniq_order_df2 = data.groupby('orderID', as_index=False).agg({'orderDate': 'first',
                                                            'articleID': list,
                                                            'colorCode': list,
                                                            'productGroup': list,
                                                            'quantity': np.sum,
                                                            'price': np.sum,
                                                            'rrp': np.sum,
                                                            'voucherID': 'first',
                                                            'voucherAmount': 'first',
                                                            'customerID': 'first',
                                                            'paymentMethod': 'first',
                                                            'returnQuantity': np.sum})
uniq_order_df2.head()

In [ ]:
#add a binary return variable
uniq_order_df2['bin_return'] = uniq_order_df2['returnQuantity'].apply(lambda x: int(x!=0))
uniq_order_df2.head()

In [ ]:
# make a copy and save as a csv
order_df = uniq_order_df2.copy()
# order_df.to_csv('fashion_train_order_df.csv')

In [ ]:
order_df.head()

In [ ]:
order_df.info()

In [ ]:
# plotting the distribution of returned quantity of items
plt.figure(figsize=(10,6))
order_df['returnQuantity'].plot.hist(bins=50)
plt.title('Distribution of Returned Quantity of Items per Order')
plt.xlabel('Items Returned per Order')
plt.ylabel('Count of Orders')
plt.show()

# Making the "customers_df"

In [ ]:
order_df.head()

In [ ]:
def unique_count(vec):
    sett = set()
    for elt in vec:
        sett.add(elt)
    return len(sett)

In [ ]:
customers_df = order_df.groupby(by='customerID', as_index=False).agg({'orderID': unique_count,
                                                                    'orderDate': list,
                                                                    'articleID': list,
                                                                    'colorCode': list,
                                                                    'productGroup': list,
                                                                    'quantity': np.sum,
                                                                    'price': np.sum,
                                                                    'rrp': np.sum,
                                                                    'voucherID': list,
                                                                    'voucherAmount': np.sum,
                                                                    'paymentMethod': list,
                                                                    'returnQuantity': np.sum,
                                                                    'bin_return': np.sum})

In [ ]:
customers_df.head()

In [ ]:
# rename columns
customers_df.rename(columns={'orderID':'totalOrders', 'quantity':'totalItemsPurchased',
                            'price':'totalDollarsPaid', 'rrp':'totalRRP', 'voucherAmount':'totalVoucherValue',
                            'returnQuantity':'totalItemsReturned', 'bin_return':'totalOrdersWithReturn'}, inplace = True)


In [ ]:
customers_df.head()

In [ ]:
customers_df.info()

In [ ]:
# save as a csv
# customers_df.to_csv('fashion_train_customers_df.csv')

## Finding Test Statistics

### Does payment method influence return probability?

In [ ]:
paymentMethod_count = order_df.groupby(by='paymentMethod')['orderID'].count()
paymentMethod_count

In [ ]:
returns_by_paymentMethod = order_df.groupby(by='paymentMethod')['bin_return'].sum()
returns_by_paymentMethod

In [ ]:
avg_returns_by_PM = returns_by_paymentMethod/paymentMethod_count
avg_returns_by_PM

In [ ]:
# plotting
plt.figure(figsize=(10,6))
x = avg_returns_by_PM.index
y = avg_returns_by_PM.values
plt.xticks(rotation=45)
plt.bar(x,y)
plt.title('Probabilities of Return By Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Probability')
plt.show()

### Are orders that use vouchers more or less likely to be returned?

In [ ]:
orders_used_voucher_count = order_df.loc[(order_df['voucherAmount']!=0)]['orderID'].count()
orders_no_voucher_count = order_df.loc[(order_df['voucherAmount']==0)]['orderID'].count()

In [ ]:
voucher_returns = order_df.loc[(order_df['voucherAmount']!=0)]['bin_return'].sum()
other_returns = order_df.loc[(order_df['voucherAmount']==0)]['bin_return'].sum()

In [ ]:
x1 = voucher_returns/orders_used_voucher_count
x2 = other_returns/orders_no_voucher_count

In [ ]:
# plotting
plt.figure(figsize=(10,6))
plt.bar(['voucher used', 'no voucher used'], [x1,x2])
plt.title('Probabilities of Return By Voucher Use')
plt.xlabel('Voucher Use')
plt.ylabel('Probability')
plt.show()

### Are orders of lower quantities more or less likely to be returned?


In [ ]:
# how many orders of each quantity were made
quantity_count = order_df.groupby(by='quantity')['orderID'].count()
# quantity_count

In [ ]:
# how many orders of each quantity had at least one returned item
count_returns_by_quant = uniq_order_df2.groupby(by='quantity')['bin_return'].sum()
# count_returns_by_quant

In [ ]:
# orders with at least one return / total number of orders
avg_returns_by_quant = count_returns_by_quant/quantity_count
# avg_returns_by_quant

In [ ]:
plt.figure(figsize=(10,6))
x = avg_returns_by_quant.index[:-1]
y = avg_returns_by_quant.values[:-1]
plt.bar(x,y)
plt.title('Probabilities of Return By Order Quantity')
plt.xlabel('Order Quantity')
plt.ylabel('Probability')
plt.show()